In [ ]:
# #############################################################################
# Importing necessary libraries

import numpy as np
import pandas as pd
import requests as requests
import json
from pathlib import Path
from m3inference import M3Inference
from collections import OrderedDict
import pprint
import urllib.request
import random
import re
import json
import os
from PIL import Image
import csv 
import sys
import string 
import zipfile as zf
from sklearn import metrics

In [ ]:
def extract_files(path, file):# Expanding the zip files
    files = zf.ZipFile(path+file, 'r')
    files.extractall(path+'a')
    files.close()

# Convert all the images in the RGB formate
def preprocess_images(file_name, height, width):
    im = Image.open(file_name)
    if im.size[0] + im.size[1] < 400:
        print(f'{file_name} is too small. Skip.')
        return
    resized_im = im.resize((height, width))
    resized_im.convert('RGB').save(file_name)
        

# Generate random numbers
def randomNumber(Numberlength):
    for x in range(Numberlength):
        number= random.randint(1,10001)
    return number

# Running the M3 inference and getting the gender
def M3_inference(output_path, true_gender):
#     try:
    with open('data.jsonl') as json_file:
        data = json.load(json_file)
    pred = m3.infer(data['images'])                     # see docstring for details
    disc=pprint.pprint(pred)

    with open(output_path, 'w', newline='') as output:  # output file is created
        for tup in pred.items():
            gender_conf = tup[1]['gender'] 
            gender = max(gender_conf.items(), key=operator.itemgetter(1))
            wr = csv.writer(output,quoting=csv.QUOTE_ALL)
            wr.writerow([tup[0], true_gender, gender[0]])
#     except urllib.error.HTTPError as e:
#         print (e)


# VARIABLES TO CHANGE:
path = '../../GIGIDL/Data/'
file = 'OUI_male.zip'
true_gender = 'M'

extract_files(path, file)
m3 = M3Inference()

# Running the imageson M3 and getting the output
output_file = path + file.split('.')[0] + '.csv'
with open(output_file, 'w', newline='') as myfile:             # Here output written in the file
    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    wr.writerow(['Imagename','G_Gender','M_Gender'])
files_path = path + 'a/' + file.split('.')[0]+'/'
data = {}
data['images'] = []
for Imagename in os.listdir(files_path):                        # Get the images from directory
    try:
        Image_location=files_path+Imagename # Get the image loacation of each image
        preprocess_images(Image_location, 224, 224)
        data['images'].append({
            "description":"", 
            "id": Imagename,
            "img_path": Image_location, 
            "lang": "en", 
            "name": "", 
            "screen_name": ""
        })
    except:
        print("Oops!",sys.exc_info()[0],"occured.")
        print()
with open('data.jsonl', 'w') as json_file: # json file for m3 is created  
    json.dump(data, json_file)
M3_inference(output_file, true_gender)


In [ ]:
file_f = '../../GIGIDL/Data/OUI_female.csv'
file_m = '../../GIGIDL/Data/OUI_male.csv'

female_all= pd.read_csv(file_f)
male_all= pd.read_csv(file_m)
full_data = pd.concat([female_all, male_all])# 
# True values
y_true = full_data['G_Gender']
# Predicted values
y_pred = full_data['M_Gender']

In [ ]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

#printing the metrics
metrics=metrics.classification_report(y_true, y_pred,output_dict=True)

#precision:
print('Precision:',round(metrics['weighted avg']['precision'],3))
#Recall
print('Recall:',round(metrics['weighted avg']['recall'],3))
#F1-score
print('F1-score:',round(metrics['weighted avg']['f1-score'],3))
#accuracy
print('Accuracy:',round(metrics['accuracy'],3))